In [2]:
pip install pandas dotenv requests streamlit

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
from typing import Optional

load_dotenv()
api_key =os.getenv('openFDA_API_KEY')
api_key is None

True

In [4]:
def fetch_api_data(url, drug_name: Optional[str] = None):
    try:
        payload = {}

        headers = {'api_key': api_key}
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data for {drug_name}: {e}")
        return {"results": []}


In [5]:
# Count adverse events by drug name within the date range
url = "https://api.fda.gov/drug/event.json?search=receivedate:[20200101+TO+20241231]&count=patient.drug.medicinalproduct.exact"
data = fetch_api_data(url, "No Drug")

df = pd.DataFrame(data["results"], columns=["term", "count"])
df.columns = ["Drug Name", "Adverse Event Count"]

# Clean data
df["Drug Name"] = df["Drug Name"].str.replace(r"\.$", "", regex=True)  # Remove trailing periods
df["Drug Name"] = df["Drug Name"].str.strip().str.upper()  # Standardize format
df = df.dropna(subset=["Drug Name", "Adverse Event Count"])  # Remove nulls
df["Adverse Event Count"] = pd.to_numeric(df["Adverse Event Count"], errors="coerce").fillna(0).astype(int)  # Ensure counts are integers
df = df.drop_duplicates(subset=["Drug Name"])  # Remove duplicates
df

,Drug Name,Adverse Event Count
0,ZANTAC,316588
1,DUPIXENT,266046
2,HUMIRA,213721
3,REVLIMID,152042
4,PREDNISONE,142546
...,...,...
95,JARDIANCE,32352
96,LYRICA,31794
97,ACYCLOVIR,31565
98,RANITIDINE CAPSULE,31504


In [6]:
# Count adverse events by patient age group (within date range)
# Fetch patient age data
url = "https://api.fda.gov/drug/event.json?search=receivedate:[20200101+TO+20241231]&count=patient.patientonsetage"
data = fetch_api_data(url, "No Age Data")

# Build DataFrame
df_age = pd.DataFrame(data["results"], columns=["term", "count"])
df_age.columns = ["Patient Age", "Adverse Event Count"]

# Clean data
df_age["Patient Age"] = pd.to_numeric(df_age["Patient Age"], errors="coerce")
df_age = df_age.dropna(subset=["Patient Age", "Adverse Event Count"])
df_age["Adverse Event Count"] = pd.to_numeric(df_age["Adverse Event Count"], errors="coerce").fillna(0).astype(int)
df_age = df_age[df_age["Patient Age"] > 0]  # Filter out invalid ages
df_age = df_age.drop_duplicates(subset=["Patient Age"])  # Remove duplicates

df_age


,Patient Age,Adverse Event Count
0,65,89474
1,63,85721
2,64,85506
3,70,85227
4,62,85161
...,...,...
95,95,3445
96,96,2456
97,97,1616
98,98,1079


In [7]:
# actions taken with drug data.
url = "https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20250507]&count=patient.drug.actiondrug&limit=100"
data = fetch_api_data(url, "No Year Data")
data
# df_freq = pd.DataFrame(data["results"], columns=["term", "count"])
# df_freq.columns = ["Year", "No of Drug Recalls"]

# df_freq["Year"] = pd.to_numeric(df_freq["Year"], errors="coerce")
# df_freq = df_freq.dropna(subset=["Year", "No of Drug Recalls"])
# df_freq["No of Drug Recalls"] = pd.to_numeric(df_freq["No of Drug Recalls"], errors="coerce").fillna(0).astype(int)
# df_freq = df_freq[df_freq["Year"] > 0]
# df_freq = df_freq.drop_duplicates(subset=["Year"])



# df_freq

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2025-04-28'},
 'results': [{'term': 5, 'count': 6770504},
  {'term': 1, 'count': 3603525},
  {'term': 4, 'count': 2986468},
  {'term': 6, 'count': 2160757},
  {'term': 2, 'count': 272611},
  {'term': 3, 'count': 127625}]}

In [14]:
url = "https://api.fda.gov/device/510k.json?count=clearance_type.exact&limit=100"
data = fetch_api_data(url, "No Year Data")
data

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2025-04-28'},
 'results': [{'term': 'Traditional', 'count': 153062},
  {'term': 'Special', 'count': 15486},
  {'term': 'Abbreviated', 'count': 2543},
  {'term': 'Direct', 'count': 334},
  {'term': 'Post-NSE', 'count': 110},
  {'term': 'Dual Track', 'count': 54}]}

In [15]:
url = "https://api.fda.gov/device/510k.json?count=advisory_committee.exact&limit=100"
data = fetch_api_data(url, "No Year Data")
data

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2025-04-28'},
 'results': [{'term': 'CV', 'count': 18637},
  {'term': 'OR', 'count': 16485},
  {'term': 'SU', 'count': 15651},
  {'term': 'HO', 'count': 15498},
  {'term': 'RA', 'count': 14014},
  {'term': 'CH', 'count': 12978},
  {'term': 'DE', 'count': 12337},
  {'term': 'GU', 'count': 10957},
  {'term': 'AN', 'count': 8270},
  {'term': 'MI', 'count': 6800},
  {'term': 'NE', 'count': 5823},
  {'term': 'PM', 'count': 5479},
  {'term': '', 'count': 4717},
  {'term': 'OP', 'count': 4698},
  {'term': 'OB', 'count': 4482},
  {'term': 'HE', 'count': 4168},
  {'term': 'EN', 'count': 3727},
  {'term'

In [16]:
url = "https://api.fda.gov/device/510k.json?count=state.exact&limit=100"
data = fetch_api_data(url, "No Year Data")
data

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2025-04-28'},
 'results': [{'term': '', 'count': 27127},
  {'term': 'CA', 'count': 26548},
  {'term': 'IL', 'count': 25589},
  {'term': 'MA', 'count': 9116},
  {'term': 'NJ', 'count': 7687},
  {'term': 'FL', 'count': 6686},
  {'term': 'NY', 'count': 6546},
  {'term': 'MN', 'count': 5873},
  {'term': 'PA', 'count': 5697},
  {'term': 'TX', 'count': 5533},
  {'term': 'MI', 'count': 5039},
  {'term': 'IN', 'count': 4194},
  {'term': 'OH', 'count': 3547},
  {'term': 'MD', 'count': 2630},
  {'term': 'CO', 'count': 2563},
  {'term': 'TN', 'count': 2545},
  {'term': 'WI', 'count': 2437},
  {'term': 'CT

In [17]:
url = "https://api.fda.gov/device/510k.json?count=decision_code.exact&limit=100"
data = fetch_api_data(url, "No Year Data")
data

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2025-04-28'},
 'results': [{'term': 'SESE', 'count': 168082},
  {'term': 'SN', 'count': 1397},
  {'term': 'SESK', 'count': 491},
  {'term': 'DENG', 'count': 444},
  {'term': 'SESU', 'count': 430},
  {'term': 'SEKD', 'count': 254},
  {'term': 'ST', 'count': 226},
  {'term': 'SESD', 'count': 127},
  {'term': 'SESP', 'count': 121},
  {'term': 'PT', 'count': 16},
  {'term': 'SI', 'count': 1}]}

In [18]:
url = "https://api.fda.gov/device/510k.json?count=product_code.exact&limit=100"
data = fetch_api_data(url, "No Year Data")
data

Error fetching data for No Year Data: 404 Client Error: Not Found for url: https://api.fda.gov/device/510k.json?count=product_code.exact&limit=100


{'results': []}

In [22]:
url = "https://api.fda.gov/food/enforcement.json?count=report_date&limit=100"
data = fetch_api_data(url, "No Year Data")
data

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2025-04-30'},
 'results': [{'time': '20120620', 'count': 50},
  {'time': '20120627', 'count': 109},
  {'time': '20120704', 'count': 45},
  {'time': '20120711', 'count': 43},
  {'time': '20120718', 'count': 41},
  {'time': '20120725', 'count': 8},
  {'time': '20120801', 'count': 37},
  {'time': '20120808', 'count': 33},
  {'time': '20120815', 'count': 178},
  {'time': '20120822', 'count': 13},
  {'time': '20120829', 'count': 72},
  {'time': '20120905', 'count': 95},
  {'time': '20120912', 'count': 21},
  {'time': '20120919', 'count': 160},
  {'time': '20120926', 'count': 72},
  {'time': '2012100